# Automatic Implicit Function Theorem
## Exact fit example
This is the main notebook referenced and annotated in the paper
> Dmitri Goloubentsev, Evgeny Lakshtanov, Vladimir V. Piterbarg (2021), Automatic Inverse Function Theorem

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

from autograd import grad, jacobian # pip install autograd
import autograd.numpy as np   # Thinly-wrapped version of Numpy to fascilitate autograd calcs

## Define the polynomial value function and the interpolator function



In this section we set up the problem that we will apply AAD to.

The function **spolyval(...)** gives the values of the 'stretched polynomial' at times **ts[n]** given the coefficients ***coefs*** and weights $w$.

The 'stretched polynomial' function is a (somewhat contrived but simple) example of an interpolator that fits all the points but which interpolation scheme, and the shape of the function between knots, depends on the weights $w$.

**spolyval(...)** corresponds to $\Omega(c,x,w)$ in the paper.

In [ ]:

def spolyval(coefs, ts, w):
    '''
    A 'stretched polynomial' function, a polynomial in wts,
    where wts = w[0]*ts + w[1]*ts**2.
    Wegiths w here control the shape of the function between knots.

    coefs:  polynomial coefs
    ts:     points where the function is evaluated
    w:      weights to transform ts into wts
    '''
    tsw = w[0]*ts + w[1]*ts**2
    val = 0.0
    for n in range(len(coefs)):
        val =  val + coefs[n] * tsw**n
    return val



We simulate a typical programming pattern where auxilliary variables such as $w$
come wrapped in various helpers, etc. This is not strictly necessary in this code but will be used later to illustrate important points. 

From the point of view of the paper, the overall construction, i.e. the class constructor and the **PricingHelper.spolyval(...)** method correspond to $\Omega(c,x,W(x))$ in the paper. 

In [ ]:

class PricingHelper:
    def __init__(self, w):
        self.w_ = w
        self.updatable = False
        # If w is none we link w's to xs's in a particular way 
        # to introduce the extra dependence of the result of spoly_interp 
        # on xs via w (admittedly, somewhat artificially). The actual update 
        # happens in the update(...) function that the clients are supposed 
        # to call when the xs are known.
        if w is None:
            self.updatable = True

    def update(self, xs, ts):
        '''
        Update the weights depending on the inputs ts (not used 
        in this example) and xs.
        '''
        if self.updatable:
            self.w_ = np.array([1.0, np.sum(xs**2)])
     
    def spolyval(self,c,ts):
        return spolyval(c, ts, self.w_) 



Function **spoly_interp(...)** calculates the coefs by fitting spolyval to **ts,xs** and returns the value of **spolyval** at some other point **t**.
Note how **w** is never seen inside the body of the function, all wrapped in **PricingHelper**.

Executing **spoly_interp(...)** corresponds to computing the implicit function $C(x,w)$ from the paper.

In [ ]:

def spoly_interp(xs, ts, t, pricing_helper):
    '''
    Fit a stretched polynomial to (ts,xs) and evaluate it at t
    Here pricing_helper (via pricing_helper.w_) is defining 
    the interpolation between the knots. 
    '''
    pricing_helper.update(xs,ts)
    
    def obj_f(c, x, pricing_helper = pricing_helper):
        return pricing_helper.spolyval(c, ts) - x

    x0 = np.zeros_like(ts)
    res = least_squares(lambda c : obj_f(c, xs), x0)
    c_fit = res.x
    return pricing_helper.spolyval(c_fit, t)

An example of applying **spoly_interp(...)**.

In [ ]:
# points we interpolate
ts = np.array([0.,1,2,3,4])
xs = np.array([2.,1,3,4,0])

# the point at which we evaluate our interpolator
t = 3.5 

# We can try different values of w. 'None' is the default that triggers
# the calculation w = w(x)
# 
# w(xs) for the particular xs above is equal to [1.0,30.0] so 
# we can pass them directly and it will not affect the output value of 
# spoly_interp(...) but of course will affect the gradients

# Uncomment one of these
# w_to_use = None
w_to_use = [1.0,30.0] 

# Set up the pricer helper
pricing_helper = PricingHelper(w_to_use)

# calculate the interpolated value
v = spoly_interp(xs,ts,t, pricing_helper)
print(f'value = {v}')

# plot a graph to see what the interpolation function looks like
t_fine = ts[0] + np.arange(101)/100*(ts[-1] - ts[0])
v_fine = spoly_interp(xs,ts,t_fine, pricing_helper)
plt.plot(t_fine, v_fine, '-', label = 'fitted interpolator')
plt.plot(ts,xs,'o', label = 'interpolated points')
plt.plot(t,v,'o',label = 'evaluation point')
plt.legend(loc = 'best')
plt.show()


Now calculate the gradients using bumping.

In [ ]:
eps = 1e-5
grad_bump = np.zeros_like(xs)
for n in range(len(xs)):
    x1 = xs.copy()
    x1[n] += eps
    grad_bump[n] = (spoly_interp(x1, ts, t, pricing_helper) - spoly_interp(xs, ts, t, pricing_helper))/eps
np.set_printoptions(precision=3)
print(f'gradients by bumping = {grad_bump}')

## Try autograd on poly_interp, 'differentiating' through the solver
`autograd` is a Python package that calculates the gradients at the same time as the values by overloading the Numpy operations.

In [ ]:
# this does not work as expected since least_squares is not 
# supported by autograd
def spoly_interp_for_autograd(xs,ts,t):
    return spoly_interp(xs,ts,t, pricing_helper)

spi_grad = grad(spoly_interp_for_autograd)
try:
    print(spi_grad(ts,xs,t))
except Exception as e:
    print(f'Does not work, exception: {e}')

## Modify spoly_interp to calculate the gradients to the inputs xs using the naive Implicit Function Theorem

Extend PricingHelper to calculate the potential dw/dx


In [ ]:
class PricingHelperIft(PricingHelper):
    '''
    We simulate a typical programming pattern where auxilliary variables such 
    as w come wrapped in various helpers, etc. This is not strictly necessary
    in this code but will be used later to illustrate some points.
    '''
    def __init__(self, w):
        super().__init__(w)
    
    def update(self, xs, ts):
        super().update(xs,ts)

        # Capture the gradients if w is in fact a function of x. We could call
        # autograd here but choose to code this by hand for brevity.
        if self.updatable:
            self.dw_dx_ = np.vstack((np.zeros_like(xs), 2*xs))
        else:
            self.dw_dx_ = np.zeros((2,len(xs)))
     

Modify **spoly_inter(...)** calling autograd when needed and implementing the IFT logic manually.

The variable **c_fit** corresponds to $C(x,W(x))$ in the paper. Note that this driver should be aware of the variable $w$ to calculate **dobj_dw** i.e. $\frac{\partial \Omega}{\partial w}$.

In [ ]:
def spoly_interp_ift(xs, ts, t, pricing_helper):
    '''
    This is a modification of spoly_interp() that supports gradients via 
    Naive IFT. We use autograd and need to use manual gradient manipulations
    to collect them all.
    
    The original function spoly_interp(...) fits a stretched polynomial to 
    (ts,xs) and evaluates it at t. Here pricing_helper (via pricing_helper.w_)
    is defining the interpolation between knots.
    '''

    # Update the weights w and extract the relevant gradients
    pricing_helper.update(xs,ts)
    dw_dx = pricing_helper.dw_dx_

    # The original objective function
    def obj_f(c, x, pricing_helper = pricing_helper):
        return pricing_helper.spolyval(c, ts) - x

    # We need an unwrapped version of the objective function for autograd 
    # to be able to calculate dobj_dw below.
    def obj_f_wrapper(c, x, w):
        helper_ = PricingHelper(w)
        return helper_.spolyval(c, ts) - x

    x0 = np.zeros_like(ts)
    res = least_squares(lambda c: obj_f(c,xs), x0)
    
    c_fit = res.x
    v = pricing_helper.spolyval(c_fit, t)
    # calc the gradients using IFT
    dobj_dc = jacobian(obj_f, argnum = 0)(c_fit,xs)
    dobj_dx = jacobian(obj_f, argnum = 1)(c_fit,xs)
    dc_dx = -np.linalg.lstsq(dobj_dc,dobj_dx, rcond = None)[0]

    # Calculate the gradient with respect to w. We need to keep adding
    # these for all "hidden" variables that are used in obj_f
    w = np.array(pricing_helper.w_.copy()) # a bit of a hoop here for autograd
    dobj_dw = jacobian(obj_f_wrapper, argnum = 2)(c_fit,xs,w)   
    
    dc_dw = -np.linalg.lstsq(dobj_dc,dobj_dw, rcond = None)[0]
    dc_dx += (dc_dw @ dw_dx)

    dv_dc = grad(spolyval, argnum = 0)(c_fit, t, w)
    dv_dx = dv_dc @ dc_dx

    # need to add the dw_dx contribution to the final valuation as well
    dv_dw = grad(spolyval, argnum = 2)(c_fit, t, w)
    dv_dx += dv_dw @ dw_dx

    return v, dv_dx

Calculate the gradients using naive IFT and compare to gradients by bumping calculated previously.


In [ ]:

pricing_helper = PricingHelperIft(w_to_use)
v_ift, grad_ift = spoly_interp_ift(xs,ts,t,pricing_helper)
print(f'value = {v_ift}')
print(f'gradients by ift = {grad_ift}')
print(f'gradients by bmp = {grad_bump}')
print(f'difference in gradients = {grad_ift - grad_bump}')

## Calculate the gradients using AAD + Automatic IFT

We implement the adjoints in **PricingHelper**. In a true AAD library these are generated automatically.

In [ ]:

class PricingHelperAdj(PricingHelperIft):
    def __init__(self, w):
        super().__init__(w)
     
     
    def spolyval_adj(self, c, ts, state_adj):
        '''
        Propagate the adjoints through spolyval. Normally generated 
        automatically by the AAD library.
        '''
        # make sure we accept a single float not just arrays
        ts = np.atleast_1d(ts)
        w=self.w_
        nc = len(c)
        nt = len(ts)

        # Just like in spolyval
        tsw = w[0]*ts + w[1]*ts**2

        sp_bar = state_adj['sp_bar']

        # the length of sp_bar changes depending on the number of outputs 
        # of spolyval which is given by nt, make sure we line up with the 
        # state_adj here
        if len(sp_bar) != nt:
            raise ValueError(f'sp_bar length {len(sp_bar)} is not equal to THE expected {nt}')

        # Start the adjoints with whatever is in state_adj already -- 
        # this is important
        c_bar = state_adj['c_bar']
        w_bar = state_adj['w_bar']

        # Loop over the length of the output of spolyval
        for i in range(nt):

            for n in range(nc):
                # accumulate adjoints to coefs
                c_bar[n] += tsw[i]**n * sp_bar[i]
                
                # Zero-order term has no sensitivity to w's
                if n==0: 
                  continue

                # accumulate adjoints for w's
                w_bar[0] += c[n] * n * tsw[i]**(n-1) * ts[i] * sp_bar[i]
                w_bar[1] += c[n] * n * tsw[i]**(n-1) * ts[i]**2 * sp_bar[i]

        # put adjoints back in the state_adj
        state_adj['c_bar'] = c_bar
        state_adj['w_bar'] = w_bar

Initialize the state for the adjoints.

In [ ]:
def init_state_adj(ncoefs):
    '''
    Initialize state_adj. This will be done by the AAD library.
    '''
    state_adj = {
        'sp_bar': np.array([1]),    
        'c_bar' : np.zeros(ncoefs),
        'x_bar': np.zeros(ncoefs),
        'w_bar' : np.zeros(2),
        'f_bar': np.zeros(ncoefs),
    }

    return state_adj
  


Adjoints for the objective function.

In [ ]:
def obj_f_adj(c, ts, x, helper, state_adj):
    '''
    Propagate adjoints through obj_f -- done by the AAD library
    '''
    f_bar = state_adj['f_bar']
    x_bar = state_adj['x_bar']

    state_adj['sp_bar'] = f_bar
    helper.spolyval_adj(c, ts, state_adj)
    x_bar -= f_bar
    state_adj['x_bar'] = x_bar

The main part, run **spoly_interp(...)** with AAD + AIFT.

  This is a modification of **spoly_interp()** that supports gradients via AAD + AIFT.
    Note that all the adjoint steps can be automatically derived from the valuation steps by the AAD library and there are no explicit gradient manipulations.    
    The original function **spoly_interp(...)** fits the stretched polynomial to **(ts,xs)** and evaluates it at **t**.
    Here **pricing_helper** (via **pricing_helper.w_**) is defining the interpolation between knots. 


In [ ]:
def spoly_interp_aad(xs, ts, t, pricing_helper):
    # Step 0. Initialize the state_adj
    state_adj = init_state_adj(len(ts))

    # Step 1. Update the weights w and extract the relevant gradients
    pricing_helper.update(xs,ts)
    dw_dx = pricing_helper.dw_dx_

    # The original objective function
    def obj_f(c, x, pricing_helper = pricing_helper):
        return pricing_helper.spolyval(c, ts) - x
    
    # Step 2. Fit the objective function and extract the coefs c we fit
    x0 = np.zeros_like(ts)
    res = least_squares(lambda c: obj_f(c,xs), x0)
    c_fit = res.x
    
    # Step 3. Calculate the value of spolyfit using the fitted coefs c_fit
    v = pricing_helper.spolyval(c_fit, t)

    # Gradient to coefs, known in the Newton method so no extra calcs here
    dobj_dc = jacobian(obj_f, argnum = 0)(c_fit, xs)

    # Adjoint for Step 3. I.e. propagate backwards until the call to the solver
    pricing_helper.spolyval_adj(c_fit, t, state_adj)
    c_bar = state_adj['c_bar']
    
    # Compute the correct adjoints of the objective function:
    obj_f_bar = -np.linalg.lstsq(dobj_dc.T, c_bar, rcond = None)[0]
    state_adj['f_bar'] = obj_f_bar
   
    # Adjoint for Step 2. Propagate through the objective function. Note that
    # we do not have to compute dobj_dw unlike the Naive IFT approach
    obj_f_adj(c_fit, ts, xs, pricing_helper, state_adj)
    x_bar = state_adj['x_bar']
    w_bar = state_adj['w_bar']
    
    # Adjoint for Step 1. Propagate through w=w(x)
    x_bar += w_bar @ dw_dx

    return v, x_bar


Calculate the gradients using AAD +AIFT and compare to the already computed gradients by bumping.

In [ ]:
pricing_helper = PricingHelperAdj(w_to_use)
v_aad, grad_aad = spoly_interp_aad(xs,ts,t,pricing_helper)
print(f'value = {v_aad}')
print(f'gradients by aad = {grad_aad}')
print(f'gradients by bmp = {grad_bump}')
print(f'difference in gradients = {grad_aad - grad_bump}')

## The end